In [19]:
import requests
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
import faiss
import re

In [20]:
# Nos divide el texto en chunks
def create_chunks(text, chunk_size=1000):
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    print(f"Created {len(chunks)} chunks")
    return chunks

In [21]:
def get_pdf_content(path):
    with open(path, 'rb') as file:
        reader = PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text() + '\n'
    return text.strip()

In [22]:
# Creamos el vector store
def create_vector_store(chunks):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    embeddings = model.encode(chunks)
    
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    index.add(embeddings.astype('float32'))
    
    return model, index

In [23]:
# Función de búsqueda se basa en la similitud de cosenos
def search(query, model, index, chunks, k=3):
    query_vector = model.encode([query])
    scores, indices = index.search(query_vector.astype('float32'), k)
    
    results = []
    for idx, score in zip(indices[0], scores[0]):
        results.append({
            'score': float(score),
            'content': chunks[idx][:200] + "..."
        })
    return results

In [24]:
from openai import OpenAI

client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key="Inserta tu api key"
)

def send_request(query):
    completion = client.chat.completions.create(
        model="nvidia/llama-3.1-nemotron-70b-instruct",
        messages=[{"role": "user", "content": query}],
        temperature=0.5,
        top_p=1,
        max_tokens=1024,
        stream=True
    )

    response_content = ""
    for chunk in completion:
        if chunk.choices[0].delta.content is not None:
            response_content += chunk.choices[0].delta.content

    return response_content

In [25]:
# Funcion para formatear en un prompt
def format_prompt(query, context):
    prompt = f"""
    En español: Basándote en el siguiente contexto, responde a la pregunta: "{query}"
    
    {context}
    
    Respuesta:"""
    return prompt


In [26]:
if __name__ == "__main__":
    path = "pdfs/laboratorio_Mongo_ii.pdf"
    text = get_pdf_content(path)
    
    # Creamos chunks
    chunks = create_chunks(text)
    
    # Creamos el vector store
    model, index = create_vector_store(chunks)
    
    # Realizamos la búsqueda
    query = "What is the porpuse of this github repository?"
    results = search(query, model, index, chunks)
    context = results[0].get('content')
    
    # Formatea el resultado en un prompt
    prompt = format_prompt(query, context)
    
    # Enviamos el prompt a la API
    response = send_request(prompt)
    print(response)

Created 8 chunks
Basándome en el contexto proporcionado, te respondo a la pregunta:

**"¿Cuál es el propósito de este repositorio de GitHub?"**

**Respuesta:**
El propósito de este repositorio de GitHub parece ser proporcionar un **entorno de práctica o desarrollo de Python** que interactúa con una base de datos **MongoDB**, utilizando la biblioteca **PyMongo**. El repositorio incluye:

1. **Configuración de un entorno de MongoDB** utilizando Docker, lo que facilita el despliegue y prueba de la base de datos de manera aislada.
2. **Un notebook de Python** (probablemente Jupyter Notebook) que, presumiblemente, contiene código de ejemplo o prácticas para interactuar con la base de datos MongoDB a través de PyMongo.

**Posibles usos del repositorio:**

- **Aprendizaje**: Para aquellos que desean aprender a interactuar con MongoDB utilizando PyMongo en un entorno Python.
- **Pruebas y Desarrollo**: Como un punto de partida para proyectos que requieren una base de datos NoSQL como MongoDB y